# Object Recognition using CNN

## imports 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.feature

# Load Data

In [2]:
images = np.load('trnImage.npy')
label_fine = np.load('trnLabel_fine.npy')
label_coarse = np.load('trnLabel_coarse.npy')

print(f'Images Shape: {images.shape}')
image_index = 1 # pick a specific image
image = images[:, :, :, image_index]

test_images = np.load('tstImage.npy')
test_label_fine = np.load('trnLabel_fine.npy')
test_label_coarse = np.load('trnLabel_coarse.npy')

Images Shape: (32, 32, 3, 50000)
